In [27]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [28]:
%pwd


'/Users/mselmaesen/Desktop/Mole-Detection-CNN-Project/src/model'

In [29]:
model = Sequential()

 

# adding the input layer to vgg model
vgg = VGG16(weights='imagenet',include_top=False, input_shape=(64,64,3))
model.add(vgg)

model.add(Flatten())
model.add(Dense(2, activation="softmax"))
model.summary()


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 2, 2, 512)         14714688  
_________________________________________________________________
flatten_5 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 4098      
Total params: 14,718,786
Trainable params: 14,718,786
Non-trainable params: 0
_________________________________________________________________


In [30]:
# don't train the vgg layers:
for layers in model.layers[0].layers:
    layers.trainable = False
    
# And Freeze also layer zero for good measure
model.layers[0].trainable = False

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 2, 2, 512)         14714688  
_________________________________________________________________
flatten_5 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 4098      
Total params: 14,718,786
Trainable params: 4,098
Non-trainable params: 14,714,688
_________________________________________________________________


In [31]:
from keras.preprocessing.image import ImageDataGenerator

# compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# defining the constants for the model training

BATCH_SIZE = 32
EPOCHS = 20
test_path='/Users/mselmaesen/Desktop/Mole-Detection-CNN-Project/src/preprocessing/temp/test'
train_path='/Users/mselmaesen/Desktop/Mole-Detection-CNN-Project/src/preprocessing/temp/train'


In [32]:
generator = ImageDataGenerator(
    rotation_range=90,
    rescale=1/255,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    brightness_range=[0.8,1.2],
    validation_split=0.2
)
test_generator = ImageDataGenerator(
    rescale=1/255,
)


In [33]:
train_set = generator.flow_from_directory(train_path, target_size=(64,64), batch_size=BATCH_SIZE)
test_set = test_generator.flow_from_directory(test_path, target_size=(64,64), batch_size=BATCH_SIZE)

model.fit_generator(train_set, steps_per_epoch=len(train_set.filenames)//BATCH_SIZE, epochs=EPOCHS, validation_data = test_set, validation_steps=len(test_set.filenames)//BATCH_SIZE )


Found 2610 images belonging to 2 classes.
Found 290 images belonging to 2 classes.
Epoch 1/20
81/81 [==============================] - 82s 1s/step - loss: 0.5043 - accuracy: 0.7862 - val_loss: 0.4886 - val_accuracy: 0.7812
Epoch 2/20
81/81 [==============================] - 87s 1s/step - loss: 0.4823 - accuracy: 0.7870 - val_loss: 0.4684 - val_accuracy: 0.7917
Epoch 3/20
81/81 [==============================] - 92s 1s/step - loss: 0.4625 - accuracy: 0.7984 - val_loss: 0.4659 - val_accuracy: 0.8090
Epoch 4/20
81/81 [==============================] - 92s 1s/step - loss: 0.4533 - accuracy: 0.8058 - val_loss: 0.4536 - val_accuracy: 0.7986
Epoch 5/20
81/81 [==============================] - 92s 1s/step - loss: 0.4577 - accuracy: 0.7866 - val_loss: 0.4615 - val_accuracy: 0.7951
Epoch 6/20
81/81 [==============================] - 96s 1s/step - loss: 0.4597 - accuracy: 0.7977 - val_loss: 0.4518 - val_accuracy: 0.8056
Epoch 7/20
81/81 [==============================] - 103s 1s/step - loss: 0.45

In [35]:
model.save_weights('model_transfer_learning.h5')